# Table of Contents
* [Learning Objectives](#Learning-Objectives)
	* [Preamble](#Preamble)
* [Sqlite3](#Sqlite3)
* [PostgreSQL (and DBAPI generally)](#PostgreSQL-%28and-DBAPI-generally%29)


# Learning Objectives

* Work with SQLite3 single-file databases
* Work with RDBMS's using the DBAPI standard

In [30]:
import sys
sys.path.insert(0, '../src')

## Preamble

In [33]:
# Load some data we'll use for later examples
import rdb
cowlitz_file = '../notebooks/data/cowlitz_river_wa_usgs_flow_data.rdb'
comment, header, cowlitz_data = rdb.read_rdb(cowlitz_file)

# Notice the form of this data is a list of namedtuples
print("%d rows of data" % len(cowlitz_data), end='\n----------\n')
for row in cowlitz_data[:3]:
    print(row)

11118 rows of data
----------
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-01', N_01_00060_00003='577', N_01_00060_00003_cd='A')
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-02', N_01_00060_00003='568', N_01_00060_00003_cd='A')
Row(agency_cd='USGS', site_no='14226500', datetime='1985-01-03', N_01_00060_00003='557', N_01_00060_00003_cd='A')


In [34]:
# Load some data we'll use for later examples
# Note the form of this data is a Pandas DataFrame
import pandas as pd
aapl = pd.read_csv('../notebooks/data/AAPL.csv', index_col='Date')
print("%d rows of data" % len(aapl), end='\n----------\n')
aapl[:3]

8514 rows of data
----------


,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2014-09-16,99.80,101.26,98.89,100.86,66818200,100.86
2014-09-15,102.81,103.05,101.44,101.63,61216500,101.63
2014-09-12,101.21,102.19,101.08,101.66,62626100,101.66


# Sqlite3

In [35]:
import os
try:
    os.remove('tmp/test-db')
except OSError:
    print("File already not there")
#!rm tmp/test-db

File already not there


In [39]:
import sqlite3
db = sqlite3.connect("tmp/test-db")
db.execute("create table stocks "
           "(symbol text, shares integer, price real, "
           " primary key (symbol))")
db.commit()

OperationalError: unable to open database file

In [ ]:
db.execute("insert into stocks values (?, ?, ?)", ('IBM', 50, 91.10))
db.execute("insert into stocks values (?, ?, ?)", ('AAPL', 100, 123.45))
db.commit()

In [ ]:
for row in db.execute("select * from stocks"):
    print(row)

In [ ]:
stocks = [('GOOG', 75, 380.13), ('AA', 100, 14.20), ('AIG', 124, 0.99)]
db.executemany("insert into stocks values (?, ?, ?)", stocks)
db.commit()

In [ ]:
list(db.execute("select * from stocks"))

In [ ]:
list(db.execute("select symbol, price from stocks where shares >= 100"))

In [ ]:
db.execute("insert into stocks values (?, ?, ?)", ('IBM', 100, 124.5))
db.commit()

In [37]:
%ls -l tmp/test-db

Parameter format not correct - "est-db".


In [ ]:
db.execute("CREATE TABLE cowlitz "
           "(agency_cd TEXT, site_no INTEGER, date DATE, "
           " discharge REAL, status TEXT, PRIMARY KEY (date))")
for row in cowlitz_data:
    db.execute("INSERT INTO cowlitz VALUES (?, ?, ?, ?, ?)", row)
db.commit()

In [ ]:
est = db.execute('SELECT COUNT(*) FROM cowlitz WHERE status="A:e"')
list(est)

In [ ]:
for d in db.execute('SELECT * FROM cowlitz WHERE '
                    'date >= "1988-01-01" AND date < "1988-01-10"'):
    print(d)

In [ ]:
%ls -l tmp/test-db
%ls -l $cowlitz_file

In [ ]:
# Need Pandas column names to be valid SQL column names
aapl['Adj_Close'] = aapl['Adj Close']
del aapl['Adj Close']
aapl[:3]

In [ ]:
aapl.to_sql('AAPL', db)

In [ ]:
for row in db.execute("SELECT * FROM AAPL LIMIT 10"):
    print(row)

# PostgreSQL (and DBAPI generally)

In [38]:
import psycopg2 # maybe import oracledb, mysql, db2
conn = psycopg2.connect(database='test', user='dmertz')
cursor = conn.cursor()
cursor.execute('SELECT version()')
version = cursor.fetchone()
print(version)

ImportError: No module named 'psycopg2'

In [ ]:
cursor.execute("drop table stocks")
conn.commit()

In [ ]:
cursor.execute("create table stocks "
               "(symbol text, shares integer, price real, "
               "primary key (symbol))")
conn.commit()

In [ ]:
cursor.execute("insert into stocks values (%s, %s, %s)", 
               ('IBM', 50, 91.10))
cursor.execute("insert into stocks values (%s, %s, %s)", 
               ('AAPL', 100, 123.45))
conn.commit()

In [ ]:
cursor.execute("select * from stocks;")
for row in cursor:
    print(row)

In [ ]:
stocks = [('GOOG', 75, 380.13), ('AA', 100, 14.20), ('AIG', 124, 0.99)]
cursor.executemany("insert into stocks values (%s, %s, %s)", stocks)
conn.commit()

In [ ]:
cursor.execute("select * from stocks")
list(cursor)

In [ ]:
cursor.execute("select column_name, data_type, character_maximum_length "
               "from INFORMATION_SCHEMA.COLUMNS "
               "where table_name = 'stocks'")
list(cursor)

In [ ]:
cursor.execute("select symbol, price from stocks where shares >= 100")
lots_of_shares = cursor.fetchall()
lots_of_shares

In [ ]:
try:
    cursor.execute("insert into stocks values (%s, %s, %s)", 
                   ('IBM', 100, 124.5))
finally:
    conn.rollback()

In [ ]:
try:
    cursor.execute("drop table cowlitz")
except psycopg2.ProgrammingError:
    print("Table does not exist... create below")
finally:
    conn.commit()
cursor.execute("CREATE TABLE cowlitz "
               "(agency_cd TEXT, site_no INTEGER, date DATE, "
               " discharge REAL, status TEXT, PRIMARY KEY (date))")
cursor.executemany("INSERT INTO cowlitz VALUES (%s, %s, %s, %s, %s)", 
                   cowlitz_data)
conn.commit()

In [ ]:
cursor.execute("SELECT COUNT(*) FROM cowlitz WHERE status = 'A:e'")
cursor.fetchall()

In [ ]:
cursor.execute("SELECT * FROM cowlitz WHERE "
               "date >= '1988-01-01' AND date < '1988-01-10'")
cursor.fetchall()